In [43]:
import pandas as pd
import numpy as np
import datetime as dt

In [23]:
channel_volume = pd.read_csv('Untouched_Business_case/Data/Channel_Volume.csv')
company_share = pd.read_csv('Untouched_Business_case/Data/Company_Share_GBO_unit.csv', delimiter=';')
location = pd.read_csv('Untouched_Business_case/Data/Locations.csv', delimiter=';').T
market_size = pd.read_csv('Untouched_Business_case/Data/Market_Sizes.csv')
subcategories = pd.read_csv('Untouched_Business_case/Data/Subcategories.csv')
categories= pd.read_csv('Untouched_Business_case/Data/Categories.csv')

CHANNEL VOLUME

In [ ]:
channel_volume.head()


In [ ]:
channel_volume.info()

In [ ]:
channel_volume['Year_date'].unique()

In [ ]:
channel_volume['Category'].unique()

In [ ]:
channel_volume.loc[channel_volume['Category'].isnull() == True]

LOCATION

In [ ]:
location

In [ ]:
location = location.dropna()
location.columns = location.iloc[0]
location = location.tail(-1)
location

COMPANY SHARE

In [ ]:
company_share

In [ ]:
company_share.info()

In [ ]:
company_share = company_share.dropna(axis = 0, how = 'all')
company_share

In [ ]:
company_share['Year_date'].unique()

In [ ]:
company_share.isnull().sum()

MARKET SIZE

In [6]:
market_size

,Location,Industry,Subcategory,Hierarchy_Level,Data_Type,Unit,Current_Constant,Currency_Conversion,Year,Year_date,RSP,Volume,Year_minus_2016,Year_minus_2022,Edition
0,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2016,31-Dec-16,301.000,27.000,4,-2,2022
1,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2017,31-Dec-17,318.000,28.000,5,-1,2022
2,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2018,31-Dec-18,458.000,40.000,6,0,2022
3,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2019,31-Dec-19,612.000,51.000,7,1,2022
4,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2020,31-Dec-20,774.000,62.000,8,2,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2022,31-Dec-22,6.545,1.204,10,4,2022
1217,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2023,31-Dec-23,6.628,1.214,11,5,2022
1218,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2024,31-Dec-24,6.708,1.227,12,6,2022
1219,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2025,31-Dec-25,6.793,1.240,13,7,2022


In [17]:
market_size.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Location             1221 non-null   int64  
 1   Industry             1221 non-null   object 
 2   Subcategory          1221 non-null   int64  
 3   Hierarchy_Level      1221 non-null   int64  
 4   Data_Type            1221 non-null   object 
 5   Unit                 1221 non-null   object 
 6   Current_Constant     1221 non-null   object 
 7   Currency_Conversion  1221 non-null   object 
 8   Year                 1221 non-null   int64  
 9   Year_date            1221 non-null   object 
 10  RSP                  1221 non-null   float64
 11  Volume               1221 non-null   float64
 12  Year_minus_2016      1221 non-null   int64  
 13  Year_minus_2022      1221 non-null   int64  
 14  Edition              1221 non-null   int64  
dtypes: float64(2), int64(7), object(6)
mem

In [12]:
market_size.shape

(1221, 15)

In [13]:
market_size = market_size.dropna(axis = 0, how = 'all')

In [14]:
market_size.shape

(1221, 15)

In [18]:
market_size["Currency_Conversion"].unique()

array(['Historic Fixed 2021 Exchange Rates, Forecast Fixed 2021 Exchange Rates'],
      dtype=object)

In [7]:
market_size.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Location             1221 non-null   int64  
 1   Industry             1221 non-null   object 
 2   Subcategory          1221 non-null   int64  
 3   Hierarchy_Level      1221 non-null   int64  
 4   Data_Type            1221 non-null   object 
 5   Unit                 1221 non-null   object 
 6   Current_Constant     1221 non-null   object 
 7   Currency_Conversion  1221 non-null   object 
 8   Year                 1221 non-null   int64  
 9   Year_date            1221 non-null   object 
 10  RSP                  1221 non-null   float64
 11  Volume               1221 non-null   float64
 12  Year_minus_2016      1221 non-null   int64  
 13  Year_minus_2022      1221 non-null   int64  
 14  Edition              1221 non-null   int64  
dtypes: float64(2), int64(7), object(6)
mem

In [59]:
market_size.rename(columns = {'RSP':'RSP (USD million)'}, inplace = True)
market_size.rename(columns = {'Volume':'Volume (liters)'}, inplace = True)

In [61]:
market_size["Volume"] = market_size["Volume"].astype(float)

In [62]:
market_size

,Location,Industry,Subcategory,Hierarchy_Level,Data_Type,Unit,Current_Constant,Currency_Conversion,Year,Year_date,RSP (USD million),Volume,Year_minus_2016,Year_minus_2022,Edition,Name_subcat,Name_cat
0,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2016,2016-12-31,301.000,27.000,4,-2,2022,Ale,Beer
1,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2017,2017-12-31,318.000,28.000,5,-1,2022,Ale,Beer
2,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2018,2018-12-31,458.000,40.000,6,0,2022,Ale,Beer
3,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2019,2019-12-31,612.000,51.000,7,1,2022,Ale,Beer
4,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2020,2020-12-31,774.000,62.000,8,2,2022,Ale,Beer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2022,2022-12-31,6.545,1.204,10,4,2022,Cider/Perry,Cider/Perry
1217,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2023,2023-12-31,6.628,1.214,11,5,2022,Cider/Perry,Cider/Perry
1218,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2024,2024-12-31,6.708,1.227,12,6,2022,Cider/Perry,Cider/Perry
1219,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2025,2025-12-31,6.793,1.240,13,7,2022,Cider/Perry,Cider/Perry


In [8]:
market_size['Year_date'].unique()

array(['31-Dec-16', '31-Dec-17', '31-Dec-18', '31-Dec-19', '31-Dec-20',
       '31-Dec-21', '31-Dec-22', '31-Dec-23', '31-Dec-24', '31-Dec-25',
       '31-Dec-26', '31-Dec-87', '31-Dec-66', '31-Dec-76'], dtype=object)

In [47]:
#Transforming the year string into datetime 
market_size['Year_date'] = pd.to_datetime(market_size['Year_date'])

/tmp/ipykernel_25852/3528307181.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  market_size['Year_date'] = pd.to_datetime(market_size['Year_date'])


In [56]:
#Verifying that every date is good
a = market_size['Year_date'].dt.year-market_size['Year']
a.isnull().count()

1221

In [57]:
#Verifying that all unit are the same, -> they are
market_size['Unit'].unique()

array(['USD million'], dtype=object)

In [48]:
market_size.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Location             1221 non-null   int64         
 1   Industry             1221 non-null   object        
 2   Subcategory          1221 non-null   int64         
 3   Hierarchy_Level      1221 non-null   int64         
 4   Data_Type            1221 non-null   object        
 5   Unit                 1221 non-null   object        
 6   Current_Constant     1221 non-null   object        
 7   Currency_Conversion  1221 non-null   object        
 8   Year                 1221 non-null   int64         
 9   Year_date            1221 non-null   datetime64[ns]
 10  RSP                  1221 non-null   float64       
 11  Volume               1221 non-null   float64       
 12  Year_minus_2016      1221 non-null   int64         
 13  Year_minus_2022      1221 non-nul

In [10]:
market_size.isna().sum()

Location               0
Industry               0
Subcategory            0
Hierarchy_Level        0
Data_Type              0
Unit                   0
Current_Constant       0
Currency_Conversion    0
Year                   0
Year_date              0
RSP                    0
Volume                 0
Year_minus_2016        0
Year_minus_2022        0
Edition                0
dtype: int64

In [37]:
sub_cat_merged

,id_subcat,Category,Name_subcat,Name_cat
0,1,1,Ale,Beer
1,2,1,Weissbier/Weizen/Wheat Beer,Beer
2,3,1,Flavoured/Mixed Lager,Beer
3,4,1,Standard Lager,Beer
4,5,1,Sorghum,Beer
5,6,1,Stout,Beer
6,7,1,Non Alcoholic Beer,Beer
7,8,1,Lager,Beer
8,9,1,Dark Beer,Beer
9,10,1,Premium Lager,Beer


In [38]:
market_size

,Location,Industry,Subcategory,Hierarchy_Level,Data_Type,Unit,Current_Constant,Currency_Conversion,Year,Year_date,RSP,Volume,Year_minus_2016,Year_minus_2022,Edition
0,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2016,31-Dec-16,301.000,27.000,4,-2,2022
1,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2017,31-Dec-17,318.000,28.000,5,-1,2022
2,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2018,31-Dec-18,458.000,40.000,6,0,2022
3,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2019,31-Dec-19,612.000,51.000,7,1,2022
4,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2020,31-Dec-20,774.000,62.000,8,2,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2022,31-Dec-22,6.545,1.204,10,4,2022
1217,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2023,31-Dec-23,6.628,1.214,11,5,2022
1218,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2024,31-Dec-24,6.708,1.227,12,6,2022
1219,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2025,31-Dec-25,6.793,1.240,13,7,2022


In [40]:
market_size = pd.merge(market_size, sub_cat_merged, how="left", left_on="Subcategory", right_on="id_subcat")
market_size = market_size.drop("id_subcat", axis=1)
market_size = market_size.drop("Category", axis=1)

In [46]:
market_size["Unit"].unique()

array(['USD million'], dtype=object)

In [41]:
market_size

,Location,Industry,Subcategory,Hierarchy_Level,Data_Type,Unit,Current_Constant,Currency_Conversion,Year,Year_date,RSP,Volume,Year_minus_2016,Year_minus_2022,Edition,Name_subcat,Name_cat
0,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2016,31-Dec-16,301.000,27.000,4,-2,2022,Ale,Beer
1,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2017,31-Dec-17,318.000,28.000,5,-1,2022,Ale,Beer
2,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2018,31-Dec-18,458.000,40.000,6,0,2022,Ale,Beer
3,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2019,31-Dec-19,612.000,51.000,7,1,2022,Ale,Beer
4,1,Alcoholic Drinks,1,3,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2020,31-Dec-20,774.000,62.000,8,2,2022,Ale,Beer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2022,31-Dec-22,6.545,1.204,10,4,2022,Cider/Perry,Cider/Perry
1217,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2023,31-Dec-23,6.628,1.214,11,5,2022,Cider/Perry,Cider/Perry
1218,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2024,31-Dec-24,6.708,1.227,12,6,2022,Cider/Perry,Cider/Perry
1219,7,Alcoholic Drinks,17,1,Total Value RSP,USD million,"Historic Constant 2021 Prices, Forecast Consta...","Historic Fixed 2021 Exchange Rates, Forecast F...",2025,31-Dec-25,6.793,1.240,13,7,2022,Cider/Perry,Cider/Perry


In [63]:
market_size.to_csv("market_size.csv")

SUBCATEGORIES

In [19]:
subcategories

,id,Category,Name
0,1,1,Ale
1,2,1,Weissbier/Weizen/Wheat Beer
2,3,1,Flavoured/Mixed Lager
3,4,1,Standard Lager
4,5,1,Sorghum
5,6,1,Stout
6,7,1,Non Alcoholic Beer
7,8,1,Lager
8,9,1,Dark Beer
9,10,1,Premium Lager


In [27]:
categories = categories.dropna(axis = 0, how = 'all')
categories

,id,Name
0,1,Beer
1,2,Wine
2,3,Spirits
3,4,RTDs/High-Strength Premixes
4,5,Cider/Perry


In [21]:
subcategories = subcategories.dropna(axis = 0, how = 'all')
subcategories

,id,Category,Name
0,1,1,Ale
1,2,1,Weissbier/Weizen/Wheat Beer
2,3,1,Flavoured/Mixed Lager
3,4,1,Standard Lager
4,5,1,Sorghum
5,6,1,Stout
6,7,1,Non Alcoholic Beer
7,8,1,Lager
8,9,1,Dark Beer
9,10,1,Premium Lager


In [31]:
sub_cat_merged = pd.merge(subcategories, categories, how='left', left_on="Category", right_on="id",suffixes=('_subcat', '_cat'))

In [33]:
sub_cat_merged

,id_subcat,Category,Name_subcat,id_cat,Name_cat
0,1,1,Ale,1,Beer
1,2,1,Weissbier/Weizen/Wheat Beer,1,Beer
2,3,1,Flavoured/Mixed Lager,1,Beer
3,4,1,Standard Lager,1,Beer
4,5,1,Sorghum,1,Beer
5,6,1,Stout,1,Beer
6,7,1,Non Alcoholic Beer,1,Beer
7,8,1,Lager,1,Beer
8,9,1,Dark Beer,1,Beer
9,10,1,Premium Lager,1,Beer


In [36]:
sub_cat_merged = sub_cat_merged.drop("id_cat", axis=1)